# VG - Assessment - Welliver

Greg Welliver   

In [ ]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as plticker
import seaborn as sns
sns.set()
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier
from numpy import mean
from numpy import std
from functools import partial
%matplotlib inline
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import TweedieRegressor

import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from scipy import stats

from sklearn.datasets import fetch_openml
from sklearn import linear_model, preprocessing, tree, svm, datasets, metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, auc, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score, f1_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder as OHE
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_tweedie_deviance, make_scorer


import xgboost
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import warnings
from subprocess import call
from IPython.display import Image
from datetime import datetime, timedelta, date
import os
import plotly.graph_objects as go
import itertools
from io import StringIO  
import pydotplus

# Seaborn settings
sns.set_context('notebook')
#sns.set_style("whitegrid")
#sns.set_style("white")
style_set = sns.set_style('whitegrid')
pd.options.display.float_format = '{:,}'.format
color_choice1 = 'plum'
color_choice2 = 'lightblue'
color_choice3 = 'lightgreen'
color_choice4 = 'blue'
#ax.yaxis.set_major_formatter('{x:,.0f}')
#sns.set_context('notebook', font_scale = 4)

#from mpl_toolkits.basemap import Basemap
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx

from sklearn.feature_extraction.text import CountVectorizer
from ydata_profiling import ProfileReport
from pandas.plotting import scatter_matrix

## Load the data



In [ ]:
# load the data
dfe = pd.read_csv("email.csv")
dfd = pd.read_csv("demos.csv")

# X = pd.concat([X_train, X_test])
# y = pd.concat([y_train, y_test])

# eval_set = [(X_train, y_train), (X_test, y_test)]

#df = pd.read_parquet("../Data/all_storm_data11.pqt")

First I'll look at the email dataset

In [ ]:
# check shape
dfe.shape

In [ ]:
# ensure that aren't duplicate IDs
dfe.nunique()

In [ ]:
dfe.head(10)

In [ ]:
dfe.value_counts()

##### Observations:
- It appears that when an event occurred (e.g. click_dt), the date is populated in the relevant column.  In the case of the bounced emails, it apperas a time is also tracked. Otherwise, if an event never occurs for a participant, the column shows as null.

In [ ]:
dfe.info()

In [ ]:
# lambda functions to create columns for email analytics
dfe['clicked_email'] = dfe['click_dt'].apply(lambda x: 1 if pd.notnull(x) else 0)
dfe['bounced_email'] = dfe['bounce_dt'].apply(lambda x: 1 if pd.notnull(x) else 0)
dfe['opened_email'] = dfe['open_dt'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [ ]:
dfe.head(20)

Now I'll look at the demos dataset

In [ ]:
# check shape
dfd.shape

In [ ]:
# ensure that aren't duplicate IDs
dfd.nunique()

In [ ]:
dfd.head(10)

# <font color='red'>re-run for final**</font>

profile = ProfileReport(dfd)
profile

##### Observations on dataset
- No missing values nor duplicates
- Appears to be high correlation between fund_ct and status, and call_ct and campaign.  Will investigate this and determine best path forward.
- There were some outliers that may need to be addressed (e.g. in fund_ct, and logon_ct); will check to see if removing them imporves modeling
- balance: there are three negative balances, which doesn't seem logical. Thsi could be a data issue, or an accounting nuance
- generally the features were reasonably well distributed. Not perfectly normal, but not skewed in a way that is overly concerning.
- tenure: generally shows a normal distribution, however there are some strange dips in certain tenure periods. could be caused by something like seasonal acquisition of participants?
- age: there is a peak in participants at one particular age - will need to investigate
- gender: one value is unknown - it may make sense to amend this to either male or female.  gender data is also highly imbalanced towards males.
- status: should only be values for 0 and 1, but there are 11 values that are either 2 or 3.  will need to modify this.
- campaign: In theory this variable should have been split into thirds - i.e. one-third received Email A, one-third received Email B, one-third did not receive the email.  Instead many more participants received Email B

Before addressing the issues called out above, I will merge the the two files on part_id

In [ ]:
df = dfd.merge(dfe[['click_dt', 'bounce_dt', 'open_dt', 'clicked_email', 'bounced_email', 'opened_email']], how = 'left',
                left_on = 'part_id', right_on = dfe['part_id'])


In [ ]:
# check shape to ensure merge went correctly
df.shape

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df_numeric = df[['fund_ct', 'logon_ct', 'call_ct', 'balance', 'tenure', 'age']]

In [ ]:
# use a scatter matrix to check if variables are normally distributed, as well see if there is correlation between variables
#scatter_matrix(df_numeric, figsize=(12,12));

In [ ]:
df.describe()

In [ ]:
# let's look at the accounts with the negative balances
df[df['balance'] < 0]

Interestingly, these users do share a number of characteristics.  However, there isn't enough information here to tell us what to do with these negative values, so will leave them as is for now.

Now let's look at the age feature.

In [ ]:
sns.histplot(df['age'], bins=30)
;

In [ ]:
df.age.describe()

Observations:
- The age of partipants has two peaks of concentration - one just above 44, and another right below 46.  There isn't anything to indicate we need to take action on this.

Now let's look at the gender variable

In [ ]:
df.gender.value_counts()

With approximately 3/4 of users being male (0), we can change the 2 to a male.

In [ ]:
df.head(10)

In [ ]:
df = df.replace({'gender' : {2 : 0}})

In [ ]:
df.gender.value_counts()

Now let's look at status, particularly the observations with values of 2 and 3

In [ ]:
df.status.value_counts()

In [ ]:
df[df['status'] > 1]

In [ ]:
df[df['status'] > 1].describe()

In [ ]:
df[df['status'] == 1].describe()

In [ ]:
df[df['status'] == 0].describe()

There doesn't appear to be any strong evidence to assign the observations in question to be either 0 or 1, so I will make them zeroes, since that represents about two-thirds of the values.

In [ ]:
df = df.replace({'status' : {2 : 0, 3 : 0}})

In [ ]:
df.status.value_counts()

Let's look at the tenure variable

In [ ]:
sns.histplot(df['tenure'], bins=30)
;

In [ ]:
df.tenure.value_counts()

As a sanity check on the data, I want to see if there is click data for participants that didn't receive emails.  In theory there shouldn't be any click data for participants that didn't get emails.

In [ ]:
df[df['campaign'] == 2].clicked_email.value_counts()

It looks like there are 1,119 participants that did not receive any emails (in campaign group 2), but are showing that they clicked the email.  I'll remove those observations since this is likely an error.

In [ ]:
df[df['campaign'] == 2].opened_email.value_counts()

It looks like there are 2,213 participants that did not receive any emails (in campaign group 2), but are showing that they clicked the email.  I'll remove those observations since this is likely an error. Note that some of these likely overlap with the above clicked_email group.

Additionally, I want to check if there are observations that show click data, but the user never opened the email.  This should also in theory not be possible.  Based on the below, there are two such observations.  We will also delete these.

In [ ]:
 df.loc[(df['opened_email'] == 0) & (df['clicked_email'] == 1)]

In [ ]:
df.shape

I'll now take the steps to remove these three cases from the dataset

In [ ]:
# need to delete observations where campaign == 2 and and clicked_email or open_email = 1
#df.loc[(df['campaign'] == 2) & (df['clicked_email'] == 1)]

In [ ]:
# remove observations where campaign == 2 and and clicked_email = 1
df = df[ ~((df['campaign'] == 2) & (df['clicked_email'] == 1))]


In [ ]:
# verify this worked correctly
df.shape

In [ ]:
# remove observations where campaign == 2 and and opened_email = 1
df = df[ ~((df['campaign'] == 2) & (df['opened_email'] == 1))]

In [ ]:
# verify that this worked correctly
df.shape

In [ ]:
# remove observations where opened_email == 0 and and clicked_email = 1
df = df[ ~((df['opened_email'] == 0) & (df['clicked_email'] == 1))]

In [ ]:
df.shape

In [ ]:
df_noemail = df[df['campaign'] == 2]
df_noemail

In [ ]:
#dfe[dfe['part_id'] == 37299]
dfe.loc[41763]

In [ ]:
df.call_ct.value_counts()

In [ ]:
df_numeric.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5)

In [ ]:
corrmat = df_numeric.corr()
f, ax = plt.subplots(figsize=(8, 6))
hm = sns.heatmap(round(corrmat,2), annot=True, ax=ax, cmap='magma',fmt='.2f',
                 linewidths=.05)

f.subplots_adjust(top=0.93)
;


In [ ]:
df.info()

# <font color='orange'>Modeling</font>

In [ ]:
# drop unnecessary columns
df = df.drop(['part_id', 'click_dt', 'bounce_dt', 'open_dt'], axis=1)

## Create dummy variables for categorical data types

# isolate categorical variables
categorical_features = df.select_dtypes(include=['object'])

df = pd.concat([df.drop(categorical_features, axis=1), pd.get_dummies(categorical_features)], axis=1)


In [ ]:

### Split data into training and testing subsets

# Create the X and y matrices from the dataframe
X = df.drop(columns = ['clicked_email'])
y = df['clicked_email']

# from sklearn.model_selection import train_test_split; reserve 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1234)

#X_train.isna().sum().sum()

In [ ]:
eval_set = [(X_train, y_train), (X_test, y_test)]

In [ ]:
blah blah

In [ ]:
profile = ProfileReport(df)
profile

# <font color='orange'>Logistic Regression</font>

In [ ]:
lr_clf = LogisticRegression(random_state = 23)
lr_clf.fit(X_train, y_train.values.ravel())
print("Classification Report for Training Data")
print(classification_report(y_train, lr_clf.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, lr_clf.predict(X_test)))

#### Let's also look at a confusion matrix

In [ ]:
y_pred = lr_clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels = lr_clf.classes_)
_, ax = plt.subplots()
display_cm = ConfusionMatrixDisplay(confusion_matrix = cm, 
                                    display_labels = ['0', '1'])
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(labels = ['0', '1'], fontsize = 12)
ax.set_yticklabels(labels = ['0', '1'], fontsize = 12)
plt.grid(False)
plt.rcParams.update({'font.size': 6})
display_cm.plot(ax = ax);

# <font color='orange'>Random Forest Model</font>


In [ ]:
# using 3 estimators
rf_clf = RandomForestClassifier(random_state = 123, n_jobs=-1)
rf_clf.fit(X_train, y_train.values.ravel())
y_pred = rf_clf.predict(X_test)

# let's look at classification report
print("Classification Report for Training Data")
print(classification_report(y_train, rf_clf.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, rf_clf.predict(X_test)))

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
feature_importance = rf_clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

#### Now hyperparameter tuning

In [ ]:
# set the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6, 9],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_leaf': [5, 10, 20, 50, 100]
}

In [ ]:
#markdown since it takes so long to run
# %%time
# rscv_rf = RandomizedSearchCV(RandomForestRegressor(random_state=1234),
#                                    param_grid, n_jobs=-1)
# rscv_rf.fit(X_train, y_train.values.ravel())
# print(random_search.best_estimator_)

In [ ]:
rf_reg_cv = RandomForestRegressor(max_depth=6, max_features=None, min_samples_leaf=10,
                      n_estimators=150, random_state=1234, n_jobs=-1, criterion='squared_error')
rf_reg_cv.fit(X_train, y_train.values.ravel())

y_pred = rf_reg_cv.predict(X_train)
#y_pred = rscv_rf.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = rf_reg_cv.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
feature_importance = rf_reg_cv.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
lgbm = LGBMRegressor(random_state=1234, metric='rmse')

# <font color='orange'>Light GBM</font>

In [ ]:
# using 3 estimators
lgbm = LGBMClassifier(random_state = 123, n_jobs=-1)
#lgbm.fit(X_train, y_train, early_stopping_rounds=20, eval_set=eval_set)
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)

# let's look at classification report
print("Classification Report for Training Data")
print(classification_report(y_train, rf_clf.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, rf_clf.predict(X_test)))

In [ ]:
feature_importance = lgbm.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

With this baseline, let's see how we can improve with hyperparameter tuning

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 6, 9],
    'n_estimators': [50, 100, 150],
    'num_leaves': [10, 30, 50],
    'colsample_bytree': [.3, .5, .8]
}

In [ ]:
# %%time
# markdown since it takes so long to run
# rscv_lgbm = RandomizedSearchCV(LGBMRegressor(random_state=1234),
#                                    param_grid, n_jobs=-1)
# rscv_lgbm.fit(X_train, y_train.values.ravel())
# print(random_search.best_estimator_)

In [ ]:
# define the model with hyperparamters from random search
lgbm_cv = LGBMRegressor(colsample_bytree=0.5, max_depth=9, n_estimators=150,
              num_leaves=30, random_state=1234, metric='rmse')
lgbm_cv.fit(X_train, y_train, early_stopping_rounds=20, eval_set=eval_set)

y_pred = lgbm_cv.predict(X_train)
#y_pred = rscv_lgbm.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = lgbm.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show();

In [ ]:
feature_importance = lgbm_cv.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

### cross validation

In [ ]:
#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1234, shuffle=True)

#build multiple linear regression model
#model = LGBMRegressor(random_state=1234)
model = LGBMRegressor(random_state=1234)


#use k-fold CV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error',
                         cv=cv, n_jobs=-1)

#view mean absolute error
print(f"{mean(abs(scores)):,.1f}")

first run: 

model = LGBMRegressor(colsample_bytree=0.5, max_depth=9, n_estimators=150,
              num_leaves=30, random_state=1234)



RMSE: 131,769.0


second model: 

model = LGBMRegressor(random_state=1234)


RMSE: 131,653.8


# <font color='orange'>XGBoost</font>


In [ ]:
# define the model
xgbr = XGBRegressor(random_state=1234, early_stopping_rounds=20, eval_metric='rmse')
xgbr.fit(X_train, y_train, eval_set=eval_set)

y_pred = xgbr.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = xgbr.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show();

In [ ]:
feature_importance = xgbr.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 6, 9],
    'n_estimators': [50, 100, 150],
    'colsample_bytree': [.3, .5, .8],
    'subsample': [.3, .5, .8]
}

In [ ]:
# %%time
# markdown since it takes so long to run
# rscv_xgb = RandomizedSearchCV(XGBRegressor(random_state=1234),
#                                    param_grid, n_jobs=-1)
# rscv_xgb.fit(X_train, y_train.values.ravel(), eval_set=eval_set)
# print(random_search.best_estimator_)

In [ ]:
# define the model
xgbr_cv = XGBRegressor(random_state=1234, eval_metric='rmse',
                       learning_rate=0.1, 
                       max_depth=6, 
                       n_estimators=150,
                       colsample_bytree=.8,
                       subsample=.8)
xgbr_cv.fit(X_train, y_train, eval_set=eval_set)

y_pred = xgbr_cv.predict(X_train)
#y_pred = rscv_xgb.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = xgbr_cv.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show();

In [ ]:
feature_importance = xgbr_cv.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(8,8))
plt.barh(pos, feature_importance[sorted_idx], align='center', color = 'plum')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

# <font color='orange'>Tweedie Regression</font>


Steps:
- Filter out claims with zero amount, as the severity model requires strictly positive target values.
- Correct for unreasonable observations (that might be data error) and a few exceptionally large claim amounts
    

In [ ]:
# load the data - this dataset has the zeroes eliminated
### comment out since current run doesn't contain any zeroes nor outliers
# X_train = pd.read_parquet("../Data/X_train_td.pqt")
# X_test = pd.read_parquet("../Data/X_test_td.pqt")
# y_train = pd.read_parquet("../Data/y_train_td.pqt") 
# y_test = pd.read_parquet("../Data/y_test_td.pqt")

# X = pd.concat([X_train, X_test])
# y = pd.concat([y_train, y_test])

In [ ]:
# tdrg = linear_model.TweedieRegressor(max_iter=500)
tdrg = TweedieRegressor(max_iter=500)
tdrg.fit(X_train, y_train.values.ravel())
# make predictions for test data
y_pred = tdrg.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = tdrg.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
param_grid = {
    'power': [0, 1, 2, 3],
    'max_iter': [50, 100, 150, 200, 250, 500, 1000, 1500]
}

In [ ]:
# %time
# markdown since it takes so long to run
# random_search = RandomizedSearchCV(linear_model.TweedieRegressor(),
#                                    param_grid, n_jobs=-1)
# random_search.fit(X_train, y_train.values.ravel())
# print(random_search.best_estimator_)

In [ ]:
#tdrg_cv = TweedieRegressor(link='log', power=3, max_iter=50)
# tdrg = linear_model.TweedieRegressor(max_iter=500)
tdrg_cv = TweedieRegressor(link='log', power=1, max_iter=50)
tdrg_cv.fit(X_train, y_train.values.ravel())

# make predictions for test data
y_pred = tdrg_cv.predict(X_train)
print('TRAIN:')
print('MAE: ', f"{mean_absolute_error(y_train, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_train, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_train, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_train, y_pred):,.1f}")
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("")
y_pred = tdrg_cv.predict(X_test)
print('TEST:')
print('MAE: ', f"{mean_absolute_error(y_test, y_pred):,.1f}")
print('MSE: ', f"{mean_squared_error(y_test, y_pred):,.1f}")
print('RMSE:', f"{np.sqrt(mean_squared_error(y_test, y_pred)):,.1f}")
print('MAPE: ', f"{mean_absolute_percentage_error(y_test, y_pred):,.1f}")
test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("")
print('MAE DIFF:', f"{((test_mae / train_mae) - 1):.0%}")
print('RMSE DIFF:', f"{((test_rmse / train_rmse) - 1):.0%}")

In [ ]:
plt.scatter(y_test, y_pred, color = 'blue', alpha=0.2)
plt.title('Actual Versus Predicted Values')
plt.xlabel('Actual')
plt.ylabel('Prredicted')
plt.xscale("log")
plt.yscale("log")
plt.show()

SCRATCH

In [ ]:
# # Create the fifth model
# model_mlr = sm.OLS(y_train, X_train)

# # Fit the model
# model_mlr_result = model_mlr.fit()

# # Evaluate the model
# model_mlr_result.summary()

In [ ]:
# #df = pd.DataFrame(data) 

# x = X
# y = y

# x = sm.add_constant(x)

# model = sm.OLS(y, x).fit()
# predictions = model.predict(x) 

# print_model = model.summary()
# print(print_model)

https://stackoverflow.com/questions/37144913/getting-valueerror-the-indices-for-endog-and-exog-are-not-aligned